In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF
from sklearn.model_selection import LeaveOneOut
from tqdm import tqdm_notebook as tqdm
from scipy.stats import norm
import keras
from keras import metrics
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout, Lambda, Input, Reshape, Conv2DTranspose
seed_value= 0
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


In [3]:
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
if K.backend()=='tensorflow':
    K.set_image_data_format('channels_first') 
    #K.image_data_format() == 'channels_first'

In [4]:
from keras.datasets import cifar10
(train_features, train_labels), (test_features, test_labels) = cifar10.load_data()
num_train, img_channels, img_rows, img_cols =  train_features.shape
num_test, _, _, _ =  test_features.shape
num_classes = len(np.unique(train_labels))


170500096/170498071 [==============================] - 23s 0us/step


In [5]:
from keras.utils import np_utils
np.random.seed(2019)
index = np.random.choice(50000, 500, replace=False)
X_cifar = train_features[index]/255
y_cifar = train_labels[index]
y_cifar = np_utils.to_categorical(y_cifar, num_classes)

In [8]:
kernel = RBF(length_scale=1)
gpr = GaussianProcessRegressor(kernel=kernel,alpha=0.1,random_state=0)
loo = LeaveOneOut()
error_list4 = []
for train_index, test_index in tqdm(loo.split(X_cifar),total=len(list(loo.split(X_cifar))),leave=False):
    X_train, X_test = X_cifar[train_index], X_cifar[test_index]
    y_train, y_test = y_cifar[train_index], y_cifar[test_index]
    X_train = X_train.reshape(-1, img_channels*img_rows*img_cols)
    X_test = X_test.reshape(-1, img_channels*img_rows*img_cols)
    gpr = GaussianProcessRegressor(kernel=kernel,alpha=0.1,random_state=0).fit(X_train, y_train)
    y_hat = np.argmax(gpr.predict(X_test), axis=1)
    y_true = np.argmax(y_test, axis=1)
    error = (y_hat - y_true) ** 2
    error_list4.append(error)
I_cv = sum(error_list4) / len(error_list4)
print('Cross validation error for Gaussian process regression : {}'.format(I_cv[0]))

Cross validation error for Gaussian process regression : 12.53


In [9]:
np.random.seed(2019)
index = np.random.choice(50000, 50, replace=False)
X_cifar = train_features[index]/255
y_cifar = train_labels[index]
y_cifar = np_utils.to_categorical(y_cifar, num_classes)


In [10]:
model = Sequential()
model.add(Conv2D(48, (3, 3), padding='same', input_shape=(3, 32, 32)))
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(96, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(96, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(192, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(X_cifar, y_cifar, batch_size=128, epochs=50, validation_split=0.2, verbose=2)

AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'

In [ ]:
error_list3 = []
for train_index, test_index in tqdm(loo.split(X_cifar),total=len(list(loo.split(X_cifar))),leave=False):
    X_train, X_test = X_cifar[train_index], X_cifar[test_index]
    y_train, y_test = y_cifar[train_index], y_cifar[test_index]
    np.random.seed(2019)
    model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data = (X_test, y_test), verbose=0)
    y_hat = np.argmax(model.predict(X_test), axis=1)
    y_true = np.argmax(y_test, axis=1)
    error = (y_hat - y_true) ** 2
    error_list3.append(error)
I_cv = sum(error_list3) / len(error_list3)
print('Cross validation error for Convolutional neural network: {}'.format(I_cv[0]))
